# Innehållsförteckning
- [Om Guidance](../../../../code/01.Introduce)
- [Installation](../../../../code/01.Introduce)
- [Obegränsad Generering](../../../../code/01.Introduce)
- [Tala för Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Välj](../../../../code/01.Introduce)
- [Tankekedja](../../../../code/01.Introduce)
- [JSON-generering](../../../../code/01.Introduce)
- [HTML-generering](../../../../code/01.Introduce)


# Om Guidance
Guidance är ett beprövat open-source Python-bibliotek för att styra utdata från vilken språkmodell (LM) som helst. Med ett API-anrop kan du uttrycka (i Python) de exakta programmatiska begränsningarna som modellen måste följa och generera det strukturerade resultatet i JSON, Python, HTML, SQL eller vilken struktur som helst som användningsfallet kräver.

Guidance skiljer sig från konventionella metoder för att skapa prompts. Det upprätthåller begränsningar genom att styra modellen token för token i inferenslagret, vilket ger högre kvalitet på utdata och minskar kostnader och latens med så mycket som 30–50 % vid användning för högt strukturerade scenarier.

För att lära dig mer om Guidance, besök [den offentliga repot på GitHub](https://github.com/guidance-ai/guidance) eller titta på [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) på Microsoft Build.


# Installation
1. Installera Guidance med `pip install guidance --pre`
2. Distribuera en Phi 3.5 mini-endpoint i Azure genom att gå till https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml och klicka på knappen "Deploy"
3. Spara din endpoints API-nyckel i en miljövariabel som heter `AZURE_PHI3_KEY` och URL:en i en miljövariabel som heter `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Obegränsad generering
Text kan genereras utan några begränsningar med funktionen `gen()`. Detta är samma som att använda modellen utan Guidance.

## Chattformatering
Precis som många chattmodeller förväntar sig Phi-3 meddelanden mellan en användare och en assistent i ett specifikt format. Guidance stöder Phi-3:s chattmall och hanterar chattformateringen åt dig. För att skapa chattvändningar, placera varje del av konversationen i ett block med `with user()` eller `with assistant()`. Ett block med `with system()` kan användas för att ställa in systemmeddelandet.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Tokenbesparingar
I starkt strukturerade scenarier kan Guidance hoppa över tokens och generera endast nödvändiga tokens, vilket förbättrar prestanda, ökar effektiviteten och sparar API-kostnader. Genererade tokens visas i denna notebook med en markerad bakgrund. Tvingade tokens visas utan markering och kostar lika mycket som inmatningstokens, vilka uppskattas till en tredjedel av kostnaden för utmatningstokens.

*Obs:* Det första exemplet med obehindrad generering kunde inte tvinga några tokens eftersom vi inte angav några begränsningar.


# Tala för Phi 3  
Med Guidance kan du enkelt infoga text i modellens svar. Detta kan vara användbart om du vill styra modellens output i en specifik riktning.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Begränsa med regex
I det tidigare exemplet svarade Phi 3 med uppföljande förklaringar efter att ha besvarat frågan med `Canberra`. För att begränsa modellens output till exakt ett ord kan en regex användas.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

Med regex genereras endast ordet `Canberra`.


# Välja mellan flera alternativ
När vissa möjliga val är kända kan du använda funktionen `select()` för att låta modellen välja från en lista med alternativ.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

Med `select()` genererades endast token `Kan`. Eftersom `Canberra` är det enda alternativet som möjligen kan fullborda svaret, tvingades de återstående token.


# Tankekedja
Tankekedja är en teknik som kan hjälpa till att förbättra kvaliteten på modellens output genom att uppmuntra den att bearbeta ett problem steg för steg. Vanligtvis krävs flera promptomgångar för att nå ett slutgiltigt svar. Först instrueras modellen att tänka steg för steg. Därefter promptas modellen igen för att ge det slutgiltiga svaret. Med standard-API:er för chattinferens krävs 2 API-anrop, och modellens genererade "tankekedja" debiteras två gånger – först som utgångstoken när modellen genererade den, och sedan igen som ingångstoken för det andra anropet. Med Guidance bearbetas och debiteras hela den flerstegsprocessen som en del av ett enda API-anrop, vilket minskar kostnader och fördröjning.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Generering av JSON
Guidance kan användas för att säkerställa generering av JSON som följer en JSON-schema eller pydantic-modell, såsom användarprofilschemat som visas här.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML-generering

Guidance kan också användas för att generera kod och följa de syntaktiska kraven i programmeringsspråket. I det här avsnittet kommer vi att skapa ett litet Guidance-program för att skriva mycket enkla HTML-webbsidor.

Vi kommer att dela upp webbsidan i mindre sektioner, var och en med sin egen Guidance-funktion. Dessa kombineras sedan i vår slutliga funktion för att skapa en HTML-webbsida.  
Därefter kör vi denna funktion mot en Guidance-aktiverad modell i Azure AI.

*Observera:* Detta kommer inte att vara en fullfjädrad HTML-generator; målet är att visa hur du kan skapa strukturerad output för dina specifika behov.

Vi börjar med att importera det vi behöver från Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML-webbsidor är mycket strukturerade, och vi kommer att 'tvinga' dessa delar av sidan med hjälp av Guidance.  
När vi uttryckligen kräver text från modellen, måste vi säkerställa att den inte innehåller något som kan vara en tagg - det vill säga, vi måste utesluta tecknen '<' och '>'.  


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Vi kan sedan använda denna funktion för att generera text inom en godtycklig HTML-tagg:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Nu ska vi skapa sidhuvudet.  
Som en del av detta behöver vi generera en sidtitel:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Kroppen av HTML-sidan kommer att fyllas med rubriker och stycken.  
Vi kan definiera en funktion för att göra varje:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Nu definierar vi själva HTML-kroppen.  
Detta använder `select()` med `recurse=True` för att skapa flera rubriker och stycken:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Nästa steg är funktionen som genererar den kompletta HTML-sidan.  
Vi lägger till HTML-starttaggen, skapar sedan headern, därefter body, och avslutar med att lägga till den avslutande HTML-taggen:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Vi tillhandahåller ett användarvänligt gränssnitt, som gör det möjligt för oss att:
- Ställa in temperaturen för genereringen
- Fånga den genererade sidan från Model-objektet


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Vi kan sedan skriva utdata till en fil:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

Och [se resultatet](../../../../code/01.Introduce/sample_page.html).



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller inexaktheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
